In [1]:
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json
import patatmo

In [2]:
# your patatmo connect developer credentials
credentials = json.load(open("C:\\users\\annefou\\Documents\\.netatmo.credentials"))

# create an api client
client = patatmo.api.client.NetatmoClient()
# tell the client's authentication your credentials
client.authentication.credentials = credentials
# optionally give the authentication a temporary file.
# The tokens are then stored there for later reuse,
# e.g. next time you invoke this script.
# This saves time because no new tokens have to be requested.
# New tokens are then only requested if the old ones expire.
client.authentication.tmpfile = "temp_auth.json"


In [15]:

# lat/lon outline of Hamburg/Germany
oslo_region = {
    "lat_ne" : 59.9999,
    "lat_sw" : 59.7809,
    "lon_ne" : 10.76,
    "lon_sw" : 10.70,
}

# issue the API request
oslo = client.Getpublicdata(region = oslo_region)
# convert the response to a pandas.DataFrame
print(oslo.dataframe())

                     altitude  humidity                 id   latitude  \
id                                                                      
70:ee:50:12:c3:38  209.000000      87.0  70:ee:50:12:c3:38  59.962199   
70:ee:50:04:70:20  220.000000      95.0  70:ee:50:04:70:20  59.961222   
70:ee:50:1b:0c:22  252.000000      87.0  70:ee:50:1b:0c:22  59.962722   
70:ee:50:28:ef:12  174.000000      93.0  70:ee:50:28:ef:12  59.959788   
70:ee:50:02:39:be  183.105591       NaN  70:ee:50:02:39:be  59.961331   
70:ee:50:05:65:38  208.000000     100.0  70:ee:50:05:65:38  59.964518   
70:ee:50:1b:06:76         NaN       NaN  70:ee:50:1b:06:76  59.961346   
70:ee:50:27:2b:d2         NaN      99.0  70:ee:50:27:2b:d2  59.962381   
70:ee:50:28:ad:7e  171.000000      98.0  70:ee:50:28:ad:7e  59.967889   
70:ee:50:2a:df:90         NaN      99.0  70:ee:50:2a:df:90  59.963683   
70:ee:50:05:5a:68  180.000000      97.0  70:ee:50:05:5a:68  59.967850   
70:ee:50:15:ec:58  242.000000      93.0  70:ee:50:1

In [16]:
latitudes=oslo.dataframe()['latitude']
longitudes=oslo.dataframe()['longitude']
temperatures=oslo.dataframe()['temperature']
for lat,lon,t in zip(latitudes, longitudes, temperatures):
    print(lat,lon,t)

59.9621995 10.6894028 -3.3
59.961222 10.694387 -1.9
59.9627216 10.7059413 0.0
59.959787860335 10.729260685145 -3.1
59.961331 10.726771 nan
59.9645181 10.7363399 -2.4
59.961346 10.751003 nan
59.962381 10.736948 -3.3
59.967889 10.770193 -7.6
59.963683 10.760551 -4.9
59.96784967 10.76844608 -7.1
59.968448 10.796146 -2.4
59.9605696 10.7939425 -2.9
59.9615836 10.781494 nan
60.034953 10.80468 -1.5
60.038232 10.803251 -3.3
59.970197895381 10.813395565082 -4.1
60.039672 10.805705 -3.6
60.067594 10.827687 0.2
60.0592673 10.8510214 -5.3
60.093228020301 10.836133686475 1.3
59.96063841001 10.863991635582 -0.6
59.9654245 10.8695242 -2.1
59.969892 10.874161 -4.4
60.049405846454 10.860368860852 nan
60.0502854 10.8593161 -3.3
60.049421440321 10.860154327509 nan
60.064511996118 10.864031791687 nan
60.062457 10.854722 nan
60.0599942 10.8564152 nan
60.084776467674 10.864900528033 -7.0
60.08754 10.873741 -10.0
60.089301 10.869684 -9.9
60.089932 10.865328 -6.0
59.966104918941 10.878043379407 -5.6
59.966752

In [11]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [17]:
cols = ['temperature', 'altitude', 'humidity']
geojson = df_to_geojson(oslo.dataframe(), cols)
geojson

{'features': [{'geometry': {'coordinates': [10.6894028, 59.9621995],
    'type': 'Point'},
   'properties': {'altitude': 209.0, 'humidity': 87.0, 'temperature': -3.3},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.694387, 59.961222], 'type': 'Point'},
   'properties': {'altitude': 220.0, 'humidity': 95.0, 'temperature': -1.9},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.7059413, 59.9627216], 'type': 'Point'},
   'properties': {'altitude': 252.0, 'humidity': 87.0, 'temperature': 0.0},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.729260685145, 59.959787860335],
    'type': 'Point'},
   'properties': {'altitude': 174.0, 'humidity': 93.0, 'temperature': -3.1},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.726771, 59.961331], 'type': 'Point'},
   'properties': {'altitude': 183.105591, 'humidity': nan, 'temperature': nan},
   'type': 'Feature'},
  {'geometry': {'coordinates': [10.7363399, 59.9645181], 'type': 'Point'},
   'properties': {'alt

In [18]:
# Map and label widgets
map = ipyl.Map(center=[53.88, 27.45], zoom=4)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))


layer = ipyl.GeoJSON(data=geojson)

map.add_layer(layer)


ipyw.VBox([map, label])

VBox(children=(Map(center=[53.88, 27.45], layers=(TileLayer(options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'temperature': -3.3, 'altitude': 209.0, 'humidity': 87.0}, 'geometry': {'type': 'Point', 'coordinates': [10.6894028, 59.9621995]}}, {'type': 'Feature', 'properties': {'temperature': -1.9, 'altitude': 220.0, 'humidity': 95.0}, 'geometry': {'type': 'Point', 'coordinates': [10.694387, 59.961222]}}, {'type': 'Feature', 'properties': {'temperature': 0.0, 'altitude': 252.0, 'humidity': 87.0}, 'geometry': {'type': 'Point', 'coordinates': [10.7059413, 59.9627216]}}, {'type': 'Feature', 'properties': {'temperature': -3.1, 'altitude': 174.0, 'humidity': 93.0}, 'geometry': {'type': 'Point', 'coordinates': [10.729260685145, 59.959787860335]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': 183.105591, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.726771, 59.961331]}}, {'type': 'Feature', 'properties': {'temperature': -2.4, 'altitude': 208.0, 'humidity': 100.0}, 'geometry': {'type': 'Point', 'coordinates': [10.7363399, 59.9645181]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': nan, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.751003, 59.961346]}}, {'type': 'Feature', 'properties': {'temperature': -3.3, 'altitude': nan, 'humidity': 99.0}, 'geometry': {'type': 'Point', 'coordinates': [10.736948, 59.962381]}}, {'type': 'Feature', 'properties': {'temperature': -7.6, 'altitude': 171.0, 'humidity': 98.0}, 'geometry': {'type': 'Point', 'coordinates': [10.770193, 59.967889]}}, {'type': 'Feature', 'properties': {'temperature': -4.9, 'altitude': nan, 'humidity': 99.0}, 'geometry': {'type': 'Point', 'coordinates': [10.760551, 59.963683]}}, {'type': 'Feature', 'properties': {'temperature': -7.1, 'altitude': 180.0, 'humidity': 97.0}, 'geometry': {'type': 'Point', 'coordinates': [10.76844608, 59.96784967]}}, {'type': 'Feature', 'properties': {'temperature': -2.4, 'altitude': 242.0, 'humidity': 93.0}, 'geometry': {'type': 'Point', 'coordinates': [10.796146, 59.968448]}}, {'type': 'Feature', 'properties': {'temperature': -2.9, 'altitude': 191.0, 'humidity': 72.0}, 'geometry': {'type': 'Point', 'coordinates': [10.7939425, 59.9605696]}}, {'type': 'Feature', 'properties': {'temperature': nan, 'altitude': 168.0, 'humidity': nan}, 'geometry': {'type': 'Point', 'coordinates': [10.781494, 59.9615836]}}, {'type': 'Feature', 'properties': {'temperature': -1.5, 'altitude': nan, 'humidity': 88.0}, 'geometry': {'type': 'Point', 'coordinates': [10.80468, 60.034953]}}, {'type': 'Feature', 'properties': {'temperature': -3.3, 'altitude': 324.0, 'humidity': 96.0}, 'geometry': {'type': 'Point', 'coordinates': [10.803251, 60.038232]}}, {'type': 'Feature', 'properties': {'temperature': -4.1, 'altitude': 292.0, 'humidity': 95.0}, 'geometry': {'type': 'Point', 'coordinates': [10.813395565082, 59.970197895381]}}, {'type': 'Feature', 'properties': {'temperature': -3.6, 'altitude': nan, 'humidity': 100.0}, 'geometry': {'type': 'Point', 'coordinates': [10.805705, 60.039672]}}, {'type': 'Feature', 'properties': {'temperature': 0.2, 'altitude': 383.0, 'humidity': 84.0}, 'geometry': {'type': 'Point', 'coordinates': [10.827687, 60.067594]}}, {'type': 'Feature', 'properties': {'temperature': -5.3, 'altitude': 223.0, 'humidity': 94.0}, 'geometry': {'type': 'Point', 'coordinates': [10.8510214, 60.0592673]}}, {'type': 'Feature', 'properties': {'temperature': 1.3, 'altitude': 537.0, 'humidity': 75.0}, 'geometry': {'type': 'Point', 'coordinates': [10.836133686475, 60.093228020301]}}, {'type': 'Feature', 'properties': {'temperature': -0.6, 'altitude': 243.0, 'humidity': 89.0}, 'geometry': {'type': 'Point', 'coordinates': [10.863991635582, 59.96063841001]}}, {'type': 'Feature', 'properties': {'temperature': -2.1, 'altitude': 228.0, 'humidity': 91.0}, 'geometry': {'type':